In [1]:
import nbformat
notebook_filename='./procs-postgres-query.ipynb'
nb = nbformat.read(notebook_filename, as_version=4)

In [2]:
nb.metadata.kernelspec.name

'python3'

In [3]:
tag='parameters'
for idx, cell in enumerate(nb.cells):
    if 'tags' in cell.metadata and tag in cell.metadata.tags:
        print(tag)

In [8]:
for idx, cell in enumerate(nb.cells):
    print(idx, '-'*25)
    print(cell.cell_type, cell.metadata)
    print(cell.source)

0 -------------------------
code {'ExecuteTime': {'end_time': '2020-07-15T08:34:42.165844Z', 'start_time': '2020-07-15T08:34:41.546961Z'}}
import psycopg2
from sagas.profile.connections import config

def get_vendors():
    """ query data from the vendors table """
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute("SELECT vendor_id, vendor_name FROM vendors ORDER BY vendor_name")
        print("The number of parts: ", cur.rowcount)
        row = cur.fetchone()

        while row is not None:
            print(row)
            row = cur.fetchone()

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

get_vendors()
1 -------------------------
code {'ExecuteTime': {'end_time': '2020-07-15T09:02:09.250496Z', 'start_time': '2020-07-15T09:02:09.035904Z'}}
get_vendors()
